# I. package import

In [1]:
pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.8-py3-none-any.whl size=16308 sha256=070bf4ed3ec6e782df0165a851cff6dd85e9888f2b2a96ff7842c3f1e67df0df
  Stored in directory: /root/.cache/pip/wheels/45/df/42/e8491dc0bdb283f5127adc65e3b26ca9c3b084ce11f20bb39e
Successfully built pandarallel


In [2]:
import requests
import json
import pandas as pd
import pandas_gbq as gbq
from pandarallel import pandarallel

In [ ]:
from google.colab import auth
auth.authenticate_user()

# II. get raw data

In [3]:
raw_query = """
WITH

reservation AS (
    SELECT 
        reservation_id, 
        driver_id, 
        ride_type,
        DATETIME(expected_pick_up_at, "Asia/Seoul") AS expected_pick_up_dt, 
        DATETIME(prestarted_at, "Asia/Seoul") AS prestart_dt,  
        start_lat, 
        start_lng, 
        origin_address AS start_address,
        origin_name AS start_name,
        estimate_total_fee, 
        ride_receipt_total + ride_receipt_discount_amount + ride_receipt_employee_discount_amount AS real_fee
    FROM `kr-co-vcnc-tada.tada_reservation.reservation_base` 
    WHERE prestarted_at IS NOT NULL  # 운행 성공 여부 등과 무관하게 prestart 가 없는 모든 호출예약은 효율화의 대상이 아니다.
    AND ride_type = "NXT"
    AND ride_date_kr BETWEEN "2022-02-01" AND "2022-02-28"
),

driver_location AS (
    SELECT 
        driver_id,  
        start_at,
        end_at, 
        lng_1 AS lng_seq_start, 
        lat_1 AS lat_seq_start, 
        lng_n AS lng_seq_end, 
        lat_n AS lat_seq_end, 
    FROM `kr-co-vcnc-tada.tada_ext.driver_activity_distance` 
    WHERE date_kr BETWEEN "2022-02-01" AND "2022-02-28"
    AND activity_status = "DISPATCHING" # prestart 는 실제로는 DISPATCHING 으로 전환되는 상태이다. 이 때, 이미 dispatching 중이거나 riding 중일 경우 나중에 찍힐 수 있다.
),

req_ride AS (
    SELECT 
        ride_id,
        first_call_service_type AS call_service_type ,
        session_final_status AS status,
        created_at_kr, 
        origin_lat, 
        origin_lng, 
        origin_address,
        destination_lat, 
        destination_lng,
        destination_address,
        IFNULL(estimation_original_max_cost, estimation_max_cost) AS estimate_ride_fee,
    FROM `kr-co-vcnc-tada.tada_store.ride_base` 
    WHERE date_kr BETWEEN "2022-02-01" AND "2022-02-28"
    AND session_start_at_kr = created_at_kr # 세션 중에서도 최초 라이드만 본다. 우리가 매칭 못시켜서 생기는 라이드가 존재하기 때문에 개별 수요로 보면 안됨
    AND call_service_type IN ("NXT", "NEAR_TAXI")
),

base AS (
    SELECT 
        r.* ,
        rr.*,
    FROM reservation AS r 
    LEFT JOIN req_ride AS rr 
    ON (CASE 
            WHEN rr.call_service_type = "NEAR_TAXI" THEN rr.created_at_kr BETWEEN prestart_dt AND expected_pick_up_dt 
            WHEN rr.call_service_type != "NEAR_TAXI" THEN rr.call_service_type = r.ride_type AND rr.created_at_kr BETWEEN prestart_dt AND expected_pick_up_dt
        END)
),

aggr_all AS (
    SELECT 
        b.* ,
        ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.start_at,"Asia/Seoul"), SECOND)) AS diff_by_start,
        ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.end_at,"Asia/Seoul"), SECOND)) AS diff_by_end,
        IF(ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.start_at,"Asia/Seoul"), SECOND)) <= ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.end_at,"Asia/Seoul"), SECOND)), DATETIME(dl.start_at,"Asia/Seoul"), DATETIME(dl.end_at,"Asia/Seoul")) AS approxy_driver_location_logged_at, 
        IF(ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.start_at,"Asia/Seoul"), SECOND)) <= ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.end_at,"Asia/Seoul"), SECOND)), dl.lng_seq_start, dl.lng_seq_end) AS approxy_driver_lng, 
        IF(ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.start_at,"Asia/Seoul"), SECOND)) <= ABS(DATETIME_DIFF(b.created_at_kr, DATETIME(dl.end_at,"Asia/Seoul"), SECOND)), dl.lat_seq_start, dl.lat_seq_end) AS approxy_driver_lat, 
    FROM base AS b
    LEFT JOIN driver_location AS dl ON b.driver_id = dl.driver_id 
    QUALIFY ROW_NUMBER() OVER (PARTITION BY driver_id, reservation_id, ride_id ORDER BY IF(diff_by_start <= diff_by_end, diff_by_start, diff_by_end) ) = 1
    AND ST_DISTANCE(ST_GEOGPOINT(approxy_driver_lng,approxy_driver_lat), ST_GEOGPOINT(origin_lng, origin_lat)) < 6000 # 배차 로직상 지역1,2,3 이 3,4,6km 거리의 라이드만 배차 대상으로 본다. 일단 최대 조건을 보는거니 6km 로 잡는다.
)

SELECT  
    reservation_id,
    driver_id,
    ride_type,
    expected_pick_up_dt,
    prestart_dt,
    start_lat,
    start_lng,
    start_address,
    start_name,
    estimate_total_fee,
    real_fee,
    approxy_driver_location_logged_at,
    approxy_driver_lng,
    approxy_driver_lat,
    ride_id,
    call_service_type,
    status,
    created_at_kr,
    origin_lat,
    origin_lng,
    origin_address,
    destination_lat,
    destination_lng,
    ROW_NUMBER() OVER (PARTITION BY reservation_id ORDER BY created_at_kr) AS ride_rn,
    estimate_ride_fee,
FROM aggr_all 
WHERE approxy_driver_lat IS NOT NULL
"""


In [4]:
raw = gbq.read_gbq(query = raw_query, project_id = "kr-co-vcnc-tada")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=6ysR9IQKe3g8pINPf40LqyynmGgqPj&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWiAHB-3stH9TRHah_znfgO__6W9tndbE7PZQlaMFnokm-OMT6wPeyA


Downloading: 100%|██████████| 47116/47116 [00:11<00:00, 3978.47rows/s]


In [5]:
raw.head()

,reservation_id,driver_id,ride_type,expected_pick_up_dt,prestart_dt,start_lat,start_lng,start_address,start_name,estimate_total_fee,...,call_service_type,status,created_at_kr,origin_lat,origin_lng,origin_address,destination_lat,destination_lng,ride_rn,estimate_ride_fee
0,H7NKUVU04VDOCUKO,DNX40942,NXT,2022-02-24 07:10:00,2022-02-24 05:23:23.146,37.491825,127.009576,서울 서초구 서초동 1553-5,오퓨런스빌딩 주차장,91700,...,NXT,DROPPED_OFF,2022-02-24 06:08:18,37.487686,126.998160,서울 서초구 방배동 892-20,37.508184,127.011631,1,7900.0
1,H7NKUVU04VDOCUKO,DNX40942,NXT,2022-02-24 07:10:00,2022-02-24 05:23:23.146,37.491825,127.009576,서울 서초구 서초동 1553-5,오퓨런스빌딩 주차장,91700,...,NXT,DROPPED_OFF,2022-02-24 06:09:34,37.497114,127.050757,서울 강남구 역삼동 757,37.522691,126.926802,2,18000.0
2,H7NKUVU04VDOCUKO,DNX40942,NXT,2022-02-24 07:10:00,2022-02-24 05:23:23.146,37.491825,127.009576,서울 서초구 서초동 1553-5,오퓨런스빌딩 주차장,91700,...,NXT,DROPPED_OFF,2022-02-24 06:11:42,37.490471,126.994016,서울 서초구 방배동 849-30,37.511934,127.025657,3,9800.0
3,H7NKUVU04VDOCUKO,DNX40942,NXT,2022-02-24 07:10:00,2022-02-24 05:23:23.146,37.491825,127.009576,서울 서초구 서초동 1553-5,오퓨런스빌딩 주차장,91700,...,NXT,DROPPED_OFF,2022-02-24 06:15:37,37.517715,127.041196,서울 강남구 청담동 41,37.603646,127.096759,4,19800.0
4,H7NKUVU04VDOCUKO,DNX40942,NXT,2022-02-24 07:10:00,2022-02-24 05:23:23.146,37.491825,127.009576,서울 서초구 서초동 1553-5,오퓨런스빌딩 주차장,91700,...,NXT,DROPPED_OFF,2022-02-24 06:19:30,37.496297,127.030157,서울 강남구 역삼동 826-29,37.505297,127.101595,5,11200.0


# III. Preprocessing

In [9]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [6]:
def inavi(
    start_lng, 
    start_lat, 
    end_lng, 
    end_lat, 
    wp1_lng, 
    wp1_lat, 
    wp2_lng, 
    wp2_lat, 
    coordType = "wgs84",
    option = "recommendation",
    usageType = 1,
    useAngle = True
    ) :

    appkey = "30utUTE2dUUsRFgMMzzk"
    query = f"https://api-maps.cloud.toast.com/maps/v3.0/appkeys/{appkey}/route-normal?option={option}&coordType={coordType}&usageType={usageType}&useAngle={useAngle}&startX={start_lng}&startY={start_lat}&endX={end_lng}&endY={end_lat}&v1a1X={wp1_lng}&via1Y={wp1_lat}&via2X={wp2_lng}&via2Y={wp2_lat}"

    r = requests.get(query).json()
    if r['header']['resultCode'] == 0 :
      dist = r['route']['data']['distance']
      time = r['route']['data']['spend_time']
      code = r['header']['resultCode']
      msg = r['header']['resultMessage']
    else :
      dist = ""
      time = ""
      code = r['header']['resultCode']
      msg = r['header']['resultMessage']

    return pd.Series([dist, time, code, msg])




In [13]:
data = raw.copy()

In [14]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
data[['distance', 'time', 'res_code','res_msg']] = data.parallel_apply(
    lambda x :inavi(
        x['approxy_driver_lng'],
        x['approxy_driver_lat'],
        x['origin_lng'],
        x['origin_lat'],
        x['destination_lng'],
        x['destination_lat'],
        x['start_lng'],
        x['start_lat']
        ),
        axis = 1
  )

KeyboardInterrupt: ignored

In [27]:
c

,reservation_id,driver_id,ride_type,expected_pick_up_dt,prestart_dt,start_lat,start_lng,start_address,start_name,estimate_total_fee,...,origin_lng,origin_address,destination_lat,destination_lng,ride_rn,estimate_ride_fee,distance,time,res_code,res_msg
0,H01HYUX8X6EXL6U3,DNX43506,NXT,2022-02-24 20:50:00,2022-02-24 19:00:49.243,37.558984,126.80351,서울 강서구 공항동 1373,김포공항 국내선 2층 3번 게이트,84600,...,126.875082,서울 양천구 목동 904,37.522577,126.867015,1,4100.0,24058,2340,0,


In [ ]:
appkey = "30utUTE2dUUsRFgMMzzk"
startX = "127.03030909067712"
startY = "37.503559214942044"
endX = "127.02298150156219"
endY = "37.52422180718409"
coordType = "wgs84"
option = "recommendation"